## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

## 建立模型

In [11]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(tf.random.normal([784, 98]), trainable=True, dtype=tf.float32)
        self.b1 = tf.Variable(tf.zeros([98]), trainable=True, dtype=tf.float32)
        self.W2 = tf.Variable(tf.random.normal([98, 10]), trainable=True, dtype=tf.float32)
        self.b2 = tf.Variable(tf.zeros([10]), trainable=True, dtype=tf.float32)
        
    def __call__(self, x):
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [12]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 104.50446 ; accuracy 0.1439
epoch 1 : loss 98.24834 ; accuracy 0.14193334
epoch 2 : loss 92.858734 ; accuracy 0.13985
epoch 3 : loss 88.19798 ; accuracy 0.1373
epoch 4 : loss 84.123665 ; accuracy 0.13525
epoch 5 : loss 80.5277 ; accuracy 0.13373333
epoch 6 : loss 77.31547 ; accuracy 0.13303334
epoch 7 : loss 74.41683 ; accuracy 0.13233334
epoch 8 : loss 71.79073 ; accuracy 0.13213333
epoch 9 : loss 69.40575 ; accuracy 0.13211666
epoch 10 : loss 67.23567 ; accuracy 0.1327
epoch 11 : loss 65.25285 ; accuracy 0.13361667
epoch 12 : loss 63.44335 ; accuracy 0.13513333
epoch 13 : loss 61.792263 ; accuracy 0.13661666
epoch 14 : loss 60.27851 ; accuracy 0.13826667
epoch 15 : loss 58.885124 ; accuracy 0.14011666
epoch 16 : loss 57.594635 ; accuracy 0.14275
epoch 17 : loss 56.39019 ; accuracy 0.1446
epoch 18 : loss 55.258408 ; accuracy 0.1466
epoch 19 : loss 54.188576 ; accuracy 0.14956667
epoch 20 : loss 53.171707 ; accuracy 0.1521
epoch 21 : loss 52.200672 ; accuracy 0.15506667
